In [1]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, query_engine, StorageContext, load_index_from_storage
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.memory import Memory
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction
from core.settings import CHROMA_PATH, llm, embed_model
import chromadb,os
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.retrievers import VectorIndexRetriever

c:\Users\Ds-Terminal\Desktop\rag-llamaindex-chroma\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


In [2]:
db_path = "C:/Users/khalil/Desktop/fastapi_llamaindex/vector_stores"

In [3]:
response = llm.complete("hello")
print(response)

Hello! It's great to chat with you. Is there something I can help you with today? Do you have a question, need some information, or just want to talk about something on your mind? I'm here to listen and assist you in any way I can.


In [4]:
embeddings = embed_model.get_text_embedding("hello")
print(len(embeddings))

768


In [5]:
#define and ollama embedding function and will be used by chroma when creating collections.
embed_fn =OllamaEmbeddingFunction(
    url="http://192.168.2.62:11434",
    model_name="nomic-embed-text:latest",
    )

In [6]:
CHROMA_PATH

WindowsPath('C:/Users/Ds-Terminal/Desktop/rag-llamaindex-chroma/vector_stores')

In [12]:
from llama_index.core import settings
settings.llm = llm
settings.embed_model = embed_model

In [13]:
vector_client = chromadb.PersistentClient(path=CHROMA_PATH)
vector_client.__dict__

{'_identifier': 'C:\\Users\\Ds-Terminal\\Desktop\\rag-llamaindex-chroma\\vector_stores',
 'tenant': 'default_tenant',
 'database': 'default_database',
 '_server': <chromadb.api.rust.RustBindingsAPI at 0x1f2d76d2ba0>,
 '_admin_client': <chromadb.api.client.AdminClient at 0x1f2d76f20d0>}

In [14]:
vector_client.list_collections()

[Collection(name=userf),
 Collection(name=userj),
 Collection(name=userb),
 Collection(name=icd_abcd_chat),
 Collection(name=useri),
 Collection(name=umar),
 Collection(name=hive_abcd_chat),
 Collection(name=useri_abcd_chat),
 Collection(name=userh),
 Collection(name=icd),
 Collection(name=sikandarf),
 Collection(name=yasirrrr),
 Collection(name=sikandar_abcd_chat),
 Collection(name=userd),
 Collection(name=hive),
 Collection(name=yasirr),
 Collection(name=yasirrr),
 Collection(name=userj_abcd_chat),
 Collection(name=yasirrrr_abcd_chat),
 Collection(name=userc),
 Collection(name=userg),
 Collection(name=umar_abcd_chat),
 Collection(name=userc_abcd_chat),
 Collection(name=usere),
 Collection(name=sikandarf_abcd_chat),
 Collection(name=usera)]

In [15]:
vector_client.get_collection("stringg").__dict__

NotFoundError: Collection [stringg] does not exists

In [9]:

#Load documents from a folder
user_dir = "./data/hive"
docs = SimpleDirectoryReader(user_dir).load_data()
print(f"Loaded {len(docs)} documents")

user_name = "icd"


Loaded 35 documents


In [10]:
docs[0]

Document(id_='57526b8c-d8ee-4670-ab88-f25fe917156a', embedding=None, metadata={'page_label': '1', 'file_name': 'HiveWorx_HR_Policy_Manual.pdf', 'file_path': 'c:\\Users\\Ds-Terminal\\Desktop\\rag-llamaindex-chroma\\data\\hive\\HiveWorx_HR_Policy_Manual.pdf', 'file_type': 'application/pdf', 'file_size': 1048607, 'creation_date': '2025-10-14', 'last_modified_date': '2025-10-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='                                                                                            HR POLICY MANUAL     \nPage 1 of 35 \n \n  \n  \n \n \nHUMAN RESOURCE POLICY MANUAL \n                                      Isl

In [16]:
user_name = "userj"

In [17]:
collection = vector_client.get_collection(user_name)
collection.__dict__


{'_client': <chromadb.api.rust.RustBindingsAPI at 0x1f2d76d2ba0>,
 '_model': Collection(id=UUID('11c446dd-c007-4753-9538-884a3257a242'), name='userj', configuration_json={'hnsw': {'space': 'cosine', 'ef_construction': 100, 'ef_search': 100, 'max_neighbors': 16, 'resize_factor': 1.2, 'sync_threshold': 1000}, 'spann': None, 'embedding_function': {'type': 'known', 'name': 'ollama', 'config': {'model_name': 'nomic-embed-text:latest', 'timeout': 60, 'url': 'http://192.168.2.62:11434'}}}, metadata=None, dimension=768, tenant='default_tenant', database='default_database', version=0, log_position=0),
 '_embedding_function': <chromadb.utils.embedding_functions.DefaultEmbeddingFunction at 0x1f2d5d6aa50>,
 '_data_loader': None}

In [18]:
vector_store = ChromaVectorStore.from_collection(collection=collection)
vector_store.__dict__

{'stores_text': True,
 'is_embedding_query': True,
 'flat_metadata': True,
 'collection_name': None,
 'host': None,
 'port': None,
 'ssl': False,
 'headers': None,
 'persist_dir': None,
 'collection_kwargs': {}}

In [72]:
result = vector_store._collection.query(query_texts=["misconduct"], include=["embeddings","documents"])

2025-10-16 05:26:27,770 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"


In [74]:
len(result["documents"])

1

In [77]:
len(result["documents"][0])

10

In [214]:

# 6️⃣ Create storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=CHROMA_PATH)
storage_context.__dict__




Loading llama_index.core.storage.kvstore.simple_kvstore from C:\Users\Ds-Terminal\Desktop\rag-llamaindex-chroma\vector_stores\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from C:\Users\Ds-Terminal\Desktop\rag-llamaindex-chroma\vector_stores\index_store.json.


{'docstore': <llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore at 0x1cda8627890>,
 'index_store': <llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore at 0x1cda8627200>,
 'vector_stores': {'default': ChromaVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, collection_name=None, host=None, port=None, ssl=False, headers=None, persist_dir=None, collection_kwargs={})},
 'graph_store': <llama_index.core.graph_stores.simple.SimpleGraphStore at 0x1cda86277a0>,
 'property_graph_store': None}

In [19]:
index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store
    )
print("Index loaded from existing vector store!")

Index loaded from existing vector store!


In [20]:
index.__dict__

{'_use_async': False,
 '_store_nodes_override': False,
 '_embed_model': OllamaEmbedding(model_name='nomic-embed-text:latest', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000001F2D763F4D0>, num_workers=None, embeddings_cache=None, base_url='http://192.168.2.62:11434', ollama_additional_kwargs={}, query_instruction=None, text_instruction=None),
 '_insert_batch_size': 2048,
 '_storage_context': StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x000001F2D76D2F90>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x000001F2D7A90830>, vector_stores={'default': ChromaVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, collection_name=None, host=None, port=None, ssl=False, headers=None, persist_dir=None, collection_kwargs={}), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorSt

In [ ]:
retriever = VectorIndexRetriever(index=index,similarity_top_k=10)
nodes = retriever.retrieve("how is squencing of sever sepsis caried out")

In [22]:
for node in nodes:
    print(node.node.__dict__)

{'id_': '55ca5329-f32f-4cf1-b1a4-5d3dc9e15438', 'embedding': None, 'metadata': {'page_label': '5', 'file_name': 'ICD-10-CM FY25 Guidelines October 1, 2024-pages-2 (1).pdf', 'ceo': 'true', 'manager': 'true'}, 'excluded_embed_metadata_keys': ['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], 'excluded_llm_metadata_keys': ['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1891b032-50c7-4d7e-a7a9-db2b6de36378', node_type='4', metadata={'page_label': '5', 'file_name': 'ICD-10-CM FY25 Guidelines October 1, 2024-pages-2 (1).pdf', 'ceo': 'true', 'manager': 'true'}, hash='ef701453dbe90f27e166a7e3a7e59f72e58594b1fb02137181571ce19a214962'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='7d98e711-c6a1-4a33-8c21-9bc6f97bc56b', node_type='1', metadata={'page_label': '5', 'file_name': 'ICD-10-CM FY25 Guidelines Oct

In [23]:
nodes[0].node.__dict__

{'id_': '55ca5329-f32f-4cf1-b1a4-5d3dc9e15438',
 'embedding': None,
 'metadata': {'page_label': '5',
  'file_name': 'ICD-10-CM FY25 Guidelines October 1, 2024-pages-2 (1).pdf',
  'ceo': 'true',
  'manager': 'true'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1891b032-50c7-4d7e-a7a9-db2b6de36378', node_type='4', metadata={'page_label': '5', 'file_name': 'ICD-10-CM FY25 Guidelines October 1, 2024-pages-2 (1).pdf', 'ceo': 'true', 'manager': 'true'}, hash='ef701453dbe90f27e166a7e3a7e59f72e58594b1fb02137181571ce19a214962'),
  <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='7d98e711-c6a1-4a33-8c21-9bc6f97bc56b', node_type='1', metadata={'page_label': '5', 'file_name

In [24]:
nodes[0].node.metadata

{'page_label': '5',
 'file_name': 'ICD-10-CM FY25 Guidelines October 1, 2024-pages-2 (1).pdf',
 'ceo': 'true',
 'manager': 'true'}

In [33]:
results = collection.get(where={"manager": "true"}, include=["metadatas", "documents"])


In [34]:
results

{'ids': ['92b28e62-bcb1-4ab6-ab60-5986128044c9',
  '41f28048-8996-4a54-832e-ab08a3d898c7',
  '7cd8c24e-59f5-4b27-81af-2286e532aa49',
  'd24849d0-37b4-4157-ba70-99dd54fa15c0',
  '2dbd9438-d97d-4c64-b5c1-e3314480e65c',
  'f341c075-cf29-4f8a-ac21-f295156a8236',
  'd34c3734-a245-447f-9365-85aa072804ef',
  '9d55be26-9da5-493a-8474-8ef35f4d1d11',
  '37086c1f-b331-4e63-a786-faceb01dab3d',
  'f98c50c8-c781-4ca6-afd9-d786ac3caa65',
  '7d98e711-c6a1-4a33-8c21-9bc6f97bc56b',
  '55ca5329-f32f-4cf1-b1a4-5d3dc9e15438',
  '09d8a853-3f84-4f70-abb5-5e2265ce8ba3',
  '49bfc7bf-7b46-4d69-8c16-b4f503a330e5',
  '4b718820-c5e8-458a-bcaa-236a724fdf9b',
  'e33afa81-5b66-4f9d-bee1-7b423d4991fd',
  '7c89e639-c5f1-40f6-afae-6829b96db9c2',
  '0cb6a48a-76a4-4627-acb9-1d1f5799396e',
  '0ebfd9fd-1213-4dc4-8c5a-2ba2d1f45b59',
  '8842be56-a6da-4e9d-8d46-a1c51eead81a',
  '5b55bddc-3eac-4d37-a26f-3d322c6450c8',
  'c9d62adf-e8a4-4a1d-9a43-c87999e9f6fa',
  '208c802b-5b5f-45ba-a081-9081dabe2b01',
  'f9c853b2-7cc8-48ce-bc09-

In [219]:
for node in nodes:
    print(node.node.id_)

7515e348-2c1f-4261-bd28-90e92fc93cfd
80232f59-5f1d-4181-ae23-1bdcaa8fdbf8
63b2ce7d-e172-46a3-bba1-602275e601e3
635aa813-38b6-47cb-a907-5542e996eec3
225b1800-4033-4f27-b175-7119db4b3e31


In [220]:
retriever = index.as_retriever(similarity_top_k=20)
nodes = retriever.retrieve("how is sequencing of sever synopsis handled")
nodes

2025-10-17 00:26:40,885 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"


[NodeWithScore(node=TextNode(id_='edffe913-7741-42cf-a8e1-1c04d0044c58', embedding=None, metadata={'page_label': '13', 'file_name': 'ICD-10-CM FY25 Guidelines October 1, 2024 (1).pdf', 'file_path': 'C:\\Users\\Ds-Terminal\\Desktop\\rag-llamaindex-chroma\\data\\icd\\ICD-10-CM FY25 Guidelines October 1, 2024 (1).pdf', 'file_type': 'application/pdf', 'file_size': 835361, 'creation_date': '2025-10-15', 'last_modified_date': '2025-10-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d6cd1d47-56f3-4a39-8792-4fb4faaff1fa', node_type='4', metadata={'page_label': '13', 'file_name': 'ICD-10-CM FY25 Guidelines October 1, 2024 (1).pdf', 'file_path': 'C:\\Users\\Ds-Terminal\\Desktop\\rag-llamaindex-chroma\\data\\icd

In [211]:
len(nodes)

10

In [207]:
raw = retriever._vector_store._collection.query(
    query_texts=["misconduct"],
    n_results=10,
    include=["documents"]
)
print("Chroma returned:", len(raw['documents'][0]))

2025-10-17 00:12:19,450 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"


Chroma returned: 10


In [206]:
len(nodes)

5

In [61]:
retriever.__dict__

{'_index': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x1cda337b4d0>,
 '_vector_store': ChromaVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, collection_name=None, host=None, port=None, ssl=False, headers=None, persist_dir=None, collection_kwargs={}),
 '_embed_model': OllamaEmbedding(model_name='nomic-embed-text:latest', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000001CDA0B3B750>, num_workers=None, embeddings_cache=None, base_url='http://192.168.2.62:11434', ollama_additional_kwargs={}, query_instruction=None, text_instruction=None),
 '_docstore': <llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore at 0x1cda32a5e50>,
 '_similarity_top_k': 2,
 '_vector_store_query_mode': <VectorStoreQueryMode.DEFAULT: 'default'>,
 '_alpha': None,
 '_node_ids': [],
 '_doc_ids': None,
 '_filters': None,
 '_sparse_top_k': None,
 '_hybrid_top_k': None,
 '_kwargs': {},
 'callback_manage

In [62]:
nodes[0]

NodeWithScore(node=TextNode(id_='f6a99a67-9dc8-4b4e-80e9-e728d00f2c7d', embedding=None, metadata={'page_label': '17', 'file_name': 'HiveWorx_HR_Policy_Manual.pdf', 'file_path': 'C:\\Users\\Ds-Terminal\\Desktop\\rag-llamaindex-chroma\\data\\icd\\HiveWorx_HR_Policy_Manual.pdf', 'file_type': 'application/pdf', 'file_size': 1048607, 'creation_date': '2025-10-15', 'last_modified_date': '2025-10-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ab60e883-f7dc-49a4-bc82-3b8807dd1000', node_type='4', metadata={'page_label': '17', 'file_name': 'HiveWorx_HR_Policy_Manual.pdf', 'file_path': 'C:\\Users\\Ds-Terminal\\Desktop\\rag-llamaindex-chroma\\data\\icd\\HiveWorx_HR_Policy_Manual.pdf', 'file_type': 'application/

In [173]:
index = load_index_from_storage(storage_context=storage_context)
chat_engine = index.as_chat_engine(llm=llm, chat_mode="condense_plus_context",system_prompt=("you are a helpful assistant that can answer general questions as well as quetions asked from the provided context. Please be concise."))

2025-10-16 23:34:08,093 - INFO - Loading all indices.


In [174]:
chat_engine.__dict__


{'_retriever': <llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever at 0x1cda84d11d0>,
 '_llm': Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000001CDA0B3B750>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x000001CD9BE136A0>, completion_to_prompt=<function default_completion_to_prompt at 0x000001CD9C4D1A80>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://192.168.2.62:11434', model='llama3:latest', temperature=None, context_window=8192, request_timeout=30.0, prompt_key='prompt', json_mode=False, additional_kwargs={}, is_function_calling_model=True, keep_alive=None, thinking=None),
 '_memory': ChatMemoryBuffer(chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history', token_limit=7936, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_speci

In [178]:
response = chat_engine.chat("what codes are given for sever synopsis?")


2025-10-16 23:35:13,467 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"
2025-10-16 23:35:13,469 - INFO - Condensed question: What are the codes provided in the ICD-10-CM Alphabetic Index for severe synopsis?
2025-10-16 23:35:13,524 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"
2025-10-16 23:35:22,383 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"


In [179]:
print(response)

According to the provided documents, specifically Section I.C.2. Neoplasms, I couldn't find any direct information on "sever synopsis". However, I can suggest that you might be referring to the subcategory of Neoplasms or perhaps Adverse effects, poisoning, underdosing and toxic effects (Section I.C.19).

In the ICD-10-CM Alphabetic Index, there are codes for various types of neoplasms, such as:

* Malignant neoplasm: C00-D49
* Benign neoplasm: D50-D49
* In situ neoplasm: D53-D59

Please note that these codes are not specific to "sever synopsis". If you could provide more context or clarify what you mean by "sever synopsis", I might be able to help you better.


In [ ]:
#######integrating memroy to the chat_engine ##############3

In [251]:
#create a new vector collection for the same user to store chat vectores
collection_name = user_name + "_chats"
chat_collection = vector_client.create_collection(collection_name, embedding_function=embed_fn)


In [252]:
vector_client.get_collection(collection_name).__dict__

{'_client': <chromadb.api.rust.RustBindingsAPI at 0x2a5219f5550>,
 '_model': Collection(id=UUID('40be0135-131b-4ebf-83fc-a5c619748022'), name='stringg_chats', configuration_json={'hnsw': {'space': 'cosine', 'ef_construction': 100, 'ef_search': 100, 'max_neighbors': 16, 'resize_factor': 1.2, 'sync_threshold': 1000}, 'spann': None, 'embedding_function': {'type': 'known', 'name': 'ollama', 'config': {'model_name': 'nomic-embed-text:latest', 'timeout': 60, 'url': 'http://192.168.2.62:11434'}}}, metadata=None, dimension=None, tenant='default_tenant', database='default_database', version=0, log_position=0),
 '_embedding_function': <chromadb.utils.embedding_functions.DefaultEmbeddingFunction at 0x2a5213dd850>,
 '_data_loader': None}

In [253]:
#create a vector store instance using the chat_collection
chat_vector_store = ChromaVectorStore.from_collection(chat_collection)

In [254]:
###defining static/ long term memroy
from llama_index.core.memory import StaticMemoryBlock,FactExtractionMemoryBlock,VectorMemoryBlock
memory_blocks=[
    StaticMemoryBlock(
    name="core-info",
    static_content="My name is khalil, I line in Islamabad, I work with llamaindex.",
    priority=0
),

FactExtractionMemoryBlock(
    name="extracted-info",
    llm=llm,
    max_facts=30,
    priority=1
),

VectorMemoryBlock(
    name = "vector_memory",
    vector_store=chat_vector_store,
    priority=2,
    embed_model=embed_model,
    similarity_top_k=3,
    retrieval_context_window=5,
    

),
]




In [ ]:
chat_dir = "chats"
chat_id="abc"
os.makedirs(chat_dir, exist_ok=True)

# Create user folder
user_chat_dir = os.path.join(chat_dir, "notebook_chat1")
os.makedirs(user_chat_dir, exist_ok=True)

# File path for chat
user_chat_path = os.path.join(user_chat_dir, f"{chat_id}.json")

# Create persistent SimpleChatStore
chat_store = SimpleChatStore.from_persist_path(persist_path=user_chat_path)

In [264]:
chat_store.store

{'chats': [ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='user message  chat content222...')]),
  ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')])]}

In [ ]:
from llama_index.core.llms import ChatMessage

In [220]:
chat_store.add_message(key="chats",message = ChatMessage(role="system", content="sytem some content.."))

In [169]:
chat_store.persist(persist_path=user_chat_path)


In [265]:
chat_store.store

{'chats': [ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='user message  chat content222...')]),
  ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')]),
  ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='sytem some content..')])]}

In [288]:
path_manual = 'sqlite+aiosqlite:///chats/notebook_chat/abc.sqlite3'
path = "sqlite+aiosqlite:///C:/Users/Ds-Terminal/Desktop/rag-llamaindex-chroma/chats/string/abc.sqlite3"

In [289]:
memory = Memory.from_defaults(
    session_id=user_name,
    token_limit=100,  # Normally you would set this to be closer to the LLM context window (i.e. 75,000, etc.)
    token_flush_size=20,
    chat_history_token_ratio=0.7,
    memory_blocks= memory_blocks,
    async_database_uri= path
)

In [290]:
memory.__dict__

{'token_limit': 100,
 'token_flush_size': 20,
 'chat_history_token_ratio': 0.7,
 'memory_blocks': [StaticMemoryBlock(name='core-info', description=None, priority=0, accept_short_term_memory=True, static_content=[TextBlock(block_type='text', text='My name is khalil, I line in Islamabad, I work with llamaindex.')]),
  FactExtractionMemoryBlock(name='extracted-info', description=None, priority=1, accept_short_term_memory=True, llm=Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002A5219CDA70>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x000002A51E3D0B80>, completion_to_prompt=<function default_completion_to_prompt at 0x000002A51EA4ACA0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://192.168.2.62:11434', model='llama3:latest', temperature=None, context_window=8192, request_timeout=30.0, prompt_key='prompt', json_mode=False, additional_kwargs

In [291]:
memory.get()

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='<memory>\n<core-info>\nMy name is khalil, I line in Islamabad, I work with llamaindex.\n</core-info>\n<extracted-info>\n<fact>Model names mentioned include "llama3.1" and BAAI/bge-base-en.</fact>\n</extracted-info>\n</memory>')])]

In [292]:
chat_engine_with_memory = index.as_chat_engine(llm=llm,memory = memory ,chat_mode="condense_plus_context",system_prompt=("you are a helpful assistant that can answer general questions as well as quetions asked from the provided context. Please be concise."))

In [293]:
result = await chat_engine_with_memory.achat("can YOu tell me the capital of Pakistan?")

2025-10-13 00:10:28,280 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"
2025-10-13 00:10:32,517 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"
2025-10-13 00:10:32,520 - INFO - Condensed question: Here's the rephrased standalone question:

What is the capital of Pakistan?
2025-10-13 00:10:32,652 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"
2025-10-13 00:10:35,563 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"


In [294]:
result.response

"A question that's not specifically related to the provided context! But don't worry, I'm here to help. According to my general knowledge, the capital of Pakistan is Islamabad. Would you like to know more about Islamabad or Pakistan in general?"

In [271]:
response.__dict__

{'text': "Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?",
 'additional_kwargs': {'tool_calls': [], 'thinking': None},
 'raw': {'model': 'llama3:latest',
  'created_at': '2025-10-13T09:25:35.003652639Z',
  'done': True,
  'done_reason': 'stop',
  'total_duration': 16188640513,
  'load_duration': 7120691371,
  'prompt_eval_count': 11,
  'prompt_eval_duration': 331030749,
  'eval_count': 26,
  'eval_duration': 8734816347,
  'message': Message(role='assistant', content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", thinking=None, images=None, tool_name=None, tool_calls=None),
  'usage': {'prompt_tokens': 11, 'completion_tokens': 26, 'total_tokens': 37}},
 'logprobs': None,
 'delta': None}

In [272]:
memory.__dict__

{'token_limit': 100,
 'token_flush_size': 20,
 'chat_history_token_ratio': 0.7,
 'memory_blocks': [StaticMemoryBlock(name='core-info', description=None, priority=0, accept_short_term_memory=True, static_content=[TextBlock(block_type='text', text='My name is khalil, I line in Islamabad, I work with llamaindex.')]),
  FactExtractionMemoryBlock(name='extracted-info', description=None, priority=1, accept_short_term_memory=True, llm=Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002A5219CDA70>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x000002A51E3D0B80>, completion_to_prompt=<function default_completion_to_prompt at 0x000002A51EA4ACA0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://192.168.2.62:11434', model='llama3:latest', temperature=None, context_window=8192, request_timeout=30.0, prompt_key='prompt', json_mode=False, additional_kwargs